In [1]:
import anndata
import pandas as pd
import numpy as np
import scanpy as sp
import zarr
import os

In [2]:
%%time
f = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_ss_ctxhippo_a_exon_count_matrix_v2.h5ad'
# adata = anndata.read(f, backed='r')
adata = anndata.read(f, backed=None)
adata

CPU times: user 933 ms, sys: 29.6 s, total: 30.6 s
Wall time: 30.7 s


AnnData object with n_obs × n_vars = 73347 × 45768
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [3]:
%%time
cells = adata.obs.index.values
genes = adata.var.index.values
ncells = len(cells)
ngenes = len(genes)

cellridx = np.random.choice(ncells, size=100, replace=False)
generidx = np.random.choice(ngenes, size=100, replace=False)
generidx

CPU times: user 2.89 ms, sys: 0 ns, total: 2.89 ms
Wall time: 2.74 ms


array([11448,  9110, 12077, 10878, 41896, 17276, 36093, 28771, 20131,
       20545, 29240, 45502, 42027,  5332, 32859, 14030, 22441,  6597,
       18591, 37633,  5189, 14547, 37036, 14328, 10671, 20430, 20766,
       27664,   367, 11037, 25616, 13560, 27532, 38069, 16022,  4722,
        5291, 23501, 42493, 27661, 41672, 27166,  9990,  1984, 38477,
       27870, 18191, 38084, 13321, 35655, 22835, 28225, 29229,   665,
       11333, 20062, 16772, 42059, 39824, 23189, 25381, 32068, 22945,
         212, 27422, 14880,  4861, 30911,  2289,  7380,   260, 29448,
        7222,  5232, 39691, 11986, 37654, 12621, 33688, 28960,  8887,
        9370, 20789,  3969, 17545, 18390, 35373, 13521,    86, 30673,
       25107, 15051,   221, 15728, 15795,  3628, 16481, 29823, 10219,
       18643])

# AnnData
- need to load all data into memory before random selection
- takes ~2s to select a random 100 cells
- could take ~0.1s if saved as csr.

In [4]:
# idx = np.random.choice(ncells, size=100, replace=False)
idx = np.random.choice(ngenes, size=100, replace=False)
idx2 = np.random.choice(1000, size=100, replace=False)
idx, idx2

(array([33589, 26760, 39076, 25752, 44713, 21302, 14998, 31791, 11332,
          456,  7549, 11724, 30072, 26029, 15869,  4792, 41331, 29291,
          706, 24543, 27179,  7698, 34855, 42923, 18898, 31414, 35637,
        41437, 35020, 36953, 30870,  5709,  4928,  8295,  9048, 18978,
        27990, 22454,  4005, 29952, 30668,  9334, 24242, 18958, 12522,
        17074, 39429, 29932,  3672,  5652, 35328,  6603, 42417, 17482,
        39214, 11741,  4686, 18429, 29910, 19414,  4729,  4849, 24902,
        40490, 33275, 17971, 16059, 18346, 28177, 31368, 18885, 34235,
        43936, 40903, 34405, 43653, 30388, 17237, 14181, 32714, 30063,
        26835, 44600, 32680, 39871,  9703, 12757, 21710,  1324, 40528,
        11376,  3840, 24552, 16814, 32157, 16462, 39423, 17011, 34592,
        41755]),
 array([946, 675, 213, 508, 763, 752,  79, 677,  16, 937, 872, 895, 109,
        403, 791,  84, 688, 273,  61, 335, 800, 815, 768, 189,  30,  13,
        359, 647, 346, 341, 607, 792, 587,  67, 902, 725

In [5]:
%%time
b = adata[idx]
b.X

CPU times: user 1.68 s, sys: 75.7 ms, total: 1.75 s
Wall time: 1.75 s


<100x45768 sparse matrix of type '<class 'numpy.float32'>'
	with 1001849 stored elements in Compressed Sparse Column format>

In [6]:
%%time
c = adata[:,idx]
c.X

CPU times: user 99.8 ms, sys: 2.63 ms, total: 102 ms
Wall time: 99.8 ms


<73347x100 sparse matrix of type '<class 'numpy.float32'>'
	with 1295419 stored elements in Compressed Sparse Column format>

# Zarr persistent array
- do not load all in, fast to init (must have) 
- fast to random access and retrieve 1 item (must have)
- this allows us to use PyTorch's DataLoader and its iterator and shuffled data strategy

If we want it to be faster, we may use this strategy: 
- do we need random access? or just shuffle and stream through?

In [7]:
dat_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/'
output = os.path.join(dat_dir, 'rna_ss.zarr')
print(output)
# zarr.save(output, z)
z1 = zarr.open(
          output, 
          mode='w', 
          shape=(ncells, ngenes), 
          chunks=(1000,None), # chunk = 1 abolishes the difference between idx and idx2; but is still overall slower.
          dtype='i4',
    )
# z1[:] = np.array(adata.X.toarray())

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna_ss.zarr


In [8]:
%%time
z2 = zarr.open(output, mode='r')
z2

CPU times: user 526 µs, sys: 3.14 ms, total: 3.67 ms
Wall time: 2.29 ms


<zarr.core.Array (73347, 45768) int32 read-only>

In [9]:
%%time
z2.oindex[idx[0],:].shape

CPU times: user 172 µs, sys: 1.02 ms, total: 1.2 ms
Wall time: 909 µs


(45768,)

In [10]:
%%time
z2.oindex[idx,:].shape

CPU times: user 316 ms, sys: 31.3 ms, total: 347 ms
Wall time: 355 ms


(100, 45768)

In [11]:
%%time
z2.oindex[idx2,:].shape

CPU times: user 20.6 ms, sys: 11.5 ms, total: 32.2 ms
Wall time: 27.8 ms


(100, 45768)